<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/Software_anomaly_detection_%E7%9B%B8%E9%96%A2%E4%BF%82%E6%95%B026_%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')

!pip install optuna #optunaのインストール

import optuna.integration.lightgbm as lgb_tune #LightGBMのパラメータチューニング
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)
# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error



# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 210 kB 54.3 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 78 kB 5.6 MB/s 
     |████████████████████████████████| 49 kB 4.8 MB/s 
     |████████████████████████████████| 113 kB 53.8 MB/s 
     |████████████████████████████████| 150 kB 57.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=6bf56d87b60550687b200dc33443dbcfee52ea9710cf15a03a10ed2e25265d08
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
#メモリ節約用の関数
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [2]:
# data_train csvファイルの読み込み
train_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/train.zip (Unzipped Files)/train.csv',usecols=['Anomaly',
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [3]:
# data_test csvファイルの読み込み
test_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/test.zip (Unzipped Files)/test.csv',usecols=[
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [4]:
#スコア0.9279の予測データを読み込み
test_y = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded2.csv')

In [5]:
#目的変数のみを取り出し値を四捨五入
test_y=test_y["Anomaly"]
test_y=test_y.round(0).astype(int)

In [6]:
#訓練データから目的変数のみを取り出す
train_y=train_df["Anomaly"]

In [7]:
#訓練データから目的変数を除去
train_x=train_df.drop(["Anomaly"],axis=1)
test_x=test_df

In [8]:
#全データをpandasに適用
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)
train_y = pd.Series(train_y)
test_y = pd.Series(test_y)

In [9]:
#trainデータとtestデータの予測値を結合
train2_x = pd.concat([train_x, test_x],axis=0)
train2_y = pd.concat([train_y, test_y],axis=0)

In [17]:
# 学習データにget_dummiesによるone-hot encodingを行う
train2_x = pd.get_dummies(train2_x, columns=["host"])
test_x = pd.get_dummies(test_x, columns=["host"])

In [15]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train2_x))[0]
tr_x, va_x = train2_x.iloc[tr_idx], train2_x.iloc[va_idx]
tr_y, va_y = train2_y.iloc[tr_idx], train2_y.iloc[va_idx]


In [24]:
# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# ニューラルネットモデルの構築
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train2_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# アーリーストッピング
# -----------------------------------
from keras.callbacks import EarlyStopping

# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
batch_size = 128
epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred, eps=1e-7)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)

Epoch 1/50
29330/29330 [==============================] - 218s 7ms/step - loss: 0.0074 - accuracy: 0.9973 - val_loss: 0.0055 - val_accuracy: 0.9976
Epoch 2/50
29330/29330 [==============================] - 219s 7ms/step - loss: 0.0042 - accuracy: 0.9984 - val_loss: 0.0026 - val_accuracy: 0.9991
Epoch 3/50
29330/29330 [==============================] - 218s 7ms/step - loss: 0.0036 - accuracy: 0.9987 - val_loss: 0.0026 - val_accuracy: 0.9991
Epoch 4/50
29330/29330 [==============================] - 217s 7ms/step - loss: 0.0033 - accuracy: 0.9988 - val_loss: 0.0024 - val_accuracy: 0.9992
Epoch 5/50
29330/29330 [==============================] - 216s 7ms/step - loss: 0.0032 - accuracy: 0.9988 - val_loss: 0.0029 - val_accuracy: 0.9989
Epoch 6/50
29330/29330 [==============================] - 218s 7ms/step - loss: 0.0030 - accuracy: 0.9989 - val_loss: 0.0023 - val_accuracy: 0.9991
Epoch 7/50
29330/29330 [==============================] - 232s 8ms/step - loss: 0.0030 - accuracy: 0.9989 - val_

In [ ]:
'''
# バリデーションデータでのスコアの確認
va_pred = model.predict(x_valid)
score = log_loss(y_valid, va_pred)
print(f'logloss: {score:.8f}')
'''

In [37]:
pred= model.predict(test_x)
print(pred)

AttributeError: ignored

In [25]:
# 構築したモデルの保存
import pickle
file = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/NN_26_param_notuning_test_reloaded.pkl'
pickle.dump(model, open(file, 'wb'))

In [26]:
# 構築したモデルの読み込み
import pickle
filename = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/NN_26_param_notuning_test_reloaded.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [27]:
# submission csvファイルの読み込み
submission_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/data.zip (Unzipped Files)/sample_submission.csv')

In [28]:
id_df=submission_df.iloc[:,0]
id_df.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [29]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [34]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','Anomaly']
output_df.head()

,id,Anomaly
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
95,95,0.0
96,96,0.0
97,97,0.0
98,98,0.0


In [31]:
output_df.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/NN_26_param_notuning_test_reloaded.csv',index=False)